**Speech to text with Openai whisper**

Author: Abdulmateen Ashifa

In [ ]:
# !pip install openai
# !pip install pyaudio

In [ ]:
from openai import OpenAI
import pyaudio
import wave


In [ ]:
OPENAI_API_KEY = 'YOUR_API_KEY'

In [ ]:
class AudioRecorder:
    def __init__(self, file_path, duration=5, sample_rate=44100, channels=1, format=pyaudio.paInt16):
        self.file_path = file_path
        self.duration = duration
        self.sample_rate = sample_rate
        self.channels = channels
        self.format = format
        self.p = pyaudio.PyAudio()

    def record_and_save_audio(self):
        try:
            stream = self.p.open(format=self.format,
                                channels=self.channels,
                                rate=self.sample_rate,
                                input=True,
                                frames_per_buffer=1024)

            print("Recording...")

            frames = []

            for i in range(0, int(self.sample_rate / 1024 * self.duration)):
                data = stream.read(1024)
                frames.append(data)

            print("Recording finished.")

            stream.stop_stream()
            stream.close()

            wf = wave.open(self.file_path, 'wb')
            wf.setnchannels(self.channels)
            wf.setsampwidth(self.p.get_sample_size(self.format))
            wf.setframerate(self.sample_rate)
            wf.writeframes(b''.join(frames))
            wf.close()

            print(f"Audio saved to {self.file_path}")

        except Exception as e:
            print(f"An error occurred: {e}")

        finally:
            self.p.terminate()

file_path = "recorded_audio.wav"
recorder = AudioRecorder(file_path, duration=5)
recorder.record_and_save_audio()


In [ ]:
class SpeechTranscriber:
    def __init__(self, api_key, model="whisper-1"):
        self.client = OpenAI(api_key=api_key)
        self.model = model

    def transcribe_speech_to_text(self, audio_path):
        try:
            with open(audio_path, "rb") as audio_file:
                transcript = self.client.audio.transcriptions.create(
                    model=self.model,
                    file=audio_file
                )
                print(transcript)
        except Exception as e:
            print(f"An error occurred during transcription: {e}")



In [ ]:

transcriber = SpeechTranscriber(api_key=OPENAI_API_KEY)
transcriber.transcribe_speech_to_text(file_path)